# 1. Auto Summarization

## 1. Model Load

In [1]:
PRETRAINED_MODEL_PATH="/content/drive/MyDrive/기종설/models/cnn dm [mobilebert-uncased-ext-sum] epoch=3.ckpt"

In [2]:
!pip install pytorch_lightning rouge_score transformers torch_optimizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 29.6 MB/s 
     |████████████████████████████████| 5.5 MB 50.2 MB/s 
     |████████████████████████████████| 61 kB 503 kB/s 
     |████████████████████████████████| 125 kB 73.5 MB/s 
     |████████████████████████████████| 529 kB 72.2 MB/s 
     |████████████████████████████████| 87 kB 6.9 MB/s 
     |████████████████████████████████| 7.6 MB 60.1 MB/s 
     |████████████████████████████████| 182 kB 52.6 MB/s 
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=aa2374c7ee4266f6108f3fd356f695105690deed8c39f6bb0230cdfb1cad2d9f
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=966d894ac8859cf0e38dc50a37b2363982c2b5da70cc96122c0a42d513d273a0
  Stored in directory: 

In [3]:
from drive.MyDrive.기종설.src.extractive import ExtractiveSummarizer
import torch
from torch import nn
# !pip install torchsummary
from torchsummary import summary as summary_
from torch.nn import functional as F

In [4]:
ext_sum_model = ExtractiveSummarizer.load_from_checkpoint(PRETRAINED_MODEL_PATH, strict=False)

Downloading:   0%|          | 0.00/847 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/saving.py:243: UserWarning: Found keys that are in the model state dict but not in the checkpoint: ['word_embedding_model.embeddings.position_ids']
  f"Found keys that are in the model state dict but not in the checkpoint: {keys.missing_keys}"


In [5]:
from transformers import pipeline

abs_sum_model = pipeline("summarization", 'sshleifer/distilbart-cnn-6-6')

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/460M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

# 2. STS Model

In [6]:
! pip install sentence_transformers datasets openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.9 MB/s 
     |████████████████████████████████| 451 kB 57.1 MB/s 
     |████████████████████████████████| 1.3 MB 66.9 MB/s 
     |████████████████████████████████| 212 kB 78.7 MB/s 
     |████████████████████████████████| 115 kB 76.6 MB/s 
     |████████████████████████████████| 127 kB 76.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=43e3b054f2a57da2ebecf2115a6e3bf64cef737729fac61e35004b9e84f119dc
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [7]:
import math
import logging
from datetime import datetime
import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, models, LoggingHandler, losses, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [8]:
pretrained_model_path1 = '/content/drive/MyDrive/기종설/models/sbert_model_3' # 수능특강/완성 학습 epoch 3
pretrained_model_path2 = '/content/drive/MyDrive/기종설/models/sbert_model_4' # 수능특강/완성 + 외부데이터셋 학습 epoch3
pretrained_model_path3 = '/content/drive/MyDrive/기종설/models/sbert_model_5' # 수능특강/완성 + 외부데이터셋 학습 epoch2
pretrained_model_path4 = '/content/drive/MyDrive/기종설/models/sbert_model_6' # 수능특강/완성 + 외부데이터셋 학습 epoch1
pretrained_model_path5 = '/content/drive/MyDrive/기종설/models/sbert_model_7' # 수능특강/완성 + 외부데이터셋 학습 epoch2 라벨 -1~1
sts_num_epochs = 10
train_batch_size = 16

In [9]:
file_path='/content/drive/MyDrive/기종설/dataset/testset.xlsx'
test_data = pd.read_excel(file_path, engine='openpyxl', header=None)
test_data.columns = ['text', 'choice1', 'choice2', 'choice3', 'choice4', 'choice5', 'answer']

In [10]:
test_data.head()

,text,choice1,choice2,choice3,choice4,choice5,answer
0,At every step in our journey through life we e...,"For a successful life, we must make specific p...","When making an important decision, you must ju...","For better choices, we need to analyze the pos...",Be careful when you want to go short to achiev...,You must take responsibility for the results o...,"For better choices, we need to analyze the pos..."
1,Urban delivery vehicles can be adapted to bett...,"In the city, bicycles can be used as an effici...",Bicycles are preferred as an alternative to re...,Bicycles can have all the economic advantages ...,Various bicycles are being developed in line w...,Countries around the world are promoting polic...,"In the city, bicycles can be used as an effici..."
2,"An important advantage of disclosure, as oppos...",steps to make public information accessible to...,benefits of publicizing information to ensure ...,strategies for companies to increase profits i...,necessities of identifying and analyzing curre...,effects of diversified markets on reasonable c...,benefits of publicizing information to ensure ...
3,Different parts of the brain’s visual system g...,Visual Systems Never Betray Our Trust!,Secret Missions of Color-Sensitive Brain Cells,Blind Spots: What Is Still Unknown About the B...,Why Brain Cells Exemplify Nature’s Recovery Pr...,Separate and Independent: Brain Cells’ Visual ...,Separate and Independent: Brain Cells’ Visual ...
4,Becoming competent in another culture means lo...,To be good at interacting with other cultural ...,When you collaborate with other employees with...,We need to cultivate community consciousness t...,"For a smooth relationship, we need to grasp th...","To reduce cultural conflicts, communication be...",To be good at interacting with other cultural ...


In [11]:
# 임시 셀
import time

file_path='/content/drive/MyDrive/기종설/dataset/datset.csv'
train_data = pd.read_csv(file_path)

text = train_data.loc[1, 'text']
print('===========Text============')
print(text)
print('=========Ext Sum=============')
start = time.time()
print(ext_sum_model.predict(text))
end = time.time()
print(f"{end - start:.5f} sec")
print('=========Abs Sum=============')
start = time.time()
print(abs_sum_model.predict(text)[0]['summary_text'])
end = time.time()
print(f"{end - start:.5f} sec")

===========Text============
Many people believe that they are very good at detecting deception. They may even cite examples where they were correct in spotting a lie when others around them were not so sensitive. The problem with such unproved evidence is that it’s difficult to know when one is wrong in believing a lie. Unless there was independent evidence later that someone was lying, how would you discover your incorrect judgment? By definition, successful lies go undetected and there is no way of keeping track of them. In fact, across scores of experiments, average detection accuracy is only slightly better than chance. Specifically, with 50% as a baseline for chance accuracy, the average detection accuracy is approximately 55%. Furthermore, there is little or no relationship between confidence in judgments and accuracy.
=========Ext Sum=============
By definition , successful lies go undetected and there is no way of keeping track of them. In fact , across scores of experiments , 

In [12]:
# Load Embedding Model
embedding_model1 = models.Transformer(
    model_name_or_path=pretrained_model_path1, 
    max_seq_length=256,
    do_lower_case=True
)

# Only use Mean Pooling -> Pooling all token embedding vectors of sentence.
pooling_model1 = models.Pooling(
    embedding_model1.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

sts_model1 = SentenceTransformer(modules=[embedding_model1, pooling_model1])

In [13]:
embedding_model2 = models.Transformer(
    model_name_or_path=pretrained_model_path2, 
    max_seq_length=256,
    do_lower_case=True
)

pooling_model2 = models.Pooling(
    embedding_model2.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

sts_model2 = SentenceTransformer(modules=[embedding_model2, pooling_model2])

In [14]:
embedding_model3 = models.Transformer(
    model_name_or_path=pretrained_model_path3, 
    max_seq_length=256,
    do_lower_case=True
)

pooling_model3 = models.Pooling(
    embedding_model3.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

sts_model3 = SentenceTransformer(modules=[embedding_model3, pooling_model3])

In [15]:
embedding_model4 = models.Transformer(
    model_name_or_path=pretrained_model_path4, 
    max_seq_length=256,
    do_lower_case=True
)

pooling_model4 = models.Pooling(
    embedding_model4.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

sts_model4 = SentenceTransformer(modules=[embedding_model4, pooling_model4])

In [16]:
embedding_model5 = models.Transformer(
    model_name_or_path=pretrained_model_path5, 
    max_seq_length=256,
    do_lower_case=True
)

pooling_model5 = models.Pooling(
    embedding_model5.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

sts_model5 = SentenceTransformer(modules=[embedding_model5, pooling_model5])

# 3. 테스트

In [21]:
from sentence_transformers import util

def get_answer(sts_model, summary_text, passages, return_score=False):
    summary_text_embedding = sts_model.encode(summary_text, convert_to_tensor=True)
    cosine_scores = []
    for i in range(len(passages)):
        pembedding = sts_model.encode(passages[i], convert_to_tensor=True)
        cosine_scores.append(util.cos_sim(summary_text_embedding, pembedding)[0][0].cpu())
    if return_score: return np.argmax(cosine_scores), cosine_scores
    else: return np.argmax(cosine_scores)

def evaulate_answer(dataset, sum_model, sts_model, problem_id, summary=True, verbose=False):
    passages = ['choice1', 'choice2', 'choice3', 'choice4', 'choice5']
    if summary: summary_text = sum_model.predict(dataset.loc[problem_id, 'text'])
    else: summary_text = dataset.loc[problem_id, 'text']
    pred = get_answer(sts_model, summary_text, dataset.loc[problem_id, passages])
    if verbose:
        print(f"pred : {dataset.loc[problem_id, 'choice' + str(pred + 1)]} \nans : {dataset.loc[problem_id, 'answer']}")
    return dataset.loc[problem_id, 'choice' + str(pred + 1)] == dataset.loc[problem_id, 'answer']

def calculate_accuracy(dataset, sum_model, sts_model, summary=True, verbose=False):
    correct, wrong = 0, 0
    for i in dataset.index:
        result = evaulate_answer(dataset, sum_model, sts_model, i, summary, verbose)
        if result: correct += 1
        else: wrong += 1
    return correct / (correct + wrong)

def pretty_view(data, sum_model, sts_model, summary=True):
    passages = ['choice1', 'choice2', 'choice3', 'choice4', 'choice5']
    if summary: summary_text = sum_model.predict(data['text'])
    else: summary_text = data['text']
    pred, scores = get_answer(sts_model, summary_text, data[passages], True)
    print('[question]', data['text'])
    print()
    for i in range(1, 6):
        print(f'[{i}] {data["choice"+str(i)]} => similarity : {scores[i-1] * 100}%')
        if data['choice'+str(i)] == data['answer']: ans_num = i
    print(f"ans : {ans_num}, pred : {pred + 1}")

In [23]:
pretty_view(test_data.loc[0], ext_sum_model, sts_model3)

[question] At every step in our journey through life we encounter
junctions with many different pathways leading into the
distance. Each choice involves uncertainty about which path will
get you to your destination. Trusting our intuition to make the
choice often ends up with us making a suboptimal choice.
Turning the uncertainty into numbers has proved a potent way of
analyzing the paths and finding the shortcut to your destination.
The mathematical theory of probability hasn’t eliminated risk,
but it allows us to manage that risk more effectively. The strategy
is to analyze all the possible scenarios that the future holds and
then to see what proportion of them lead to success or failure.
This gives you a much better map of the future on which to base
your decisions about which path to choose.

[1] For a successful life, we must make specific plans for the future. => similarity : 19.880414962768555%
[2] When making an important decision, you must judge according to your intuition. =>

In [ ]:
acc = calculate_accuracy(test_data, ext_sum_model, sts_model3, True, True)
print(f"Accuracy : {acc * 100} %")

pred : For better choices, we need to analyze the possibility of success. 
ans : For better choices, we need to analyze the possibility of success.
pred : In the city, bicycles can be used as an efficient shipping means. 
ans : In the city, bicycles can be used as an efficient shipping means.
pred : benefits of publicizing information to ensure free choices 
ans : benefits of publicizing information to ensure free choices
pred : Separate and Independent: Brain Cells’ Visual Perceptions 
ans : Separate and Independent: Brain Cells’ Visual Perceptions
pred : For a smooth relationship, we need to grasp the commonality of each other first. 
ans : To be good at interacting with other cultural people, you need to know the other side of the culture.
pred : Economics provides theoretical basis when the tax law is revised. 
ans : It is important to set a moral goal when creating a tax law.
pred : necessity of prioritizing trialability in agricultural innovation 
ans : limitations of using empir

1. sts_model_3 + no sum_model : 45%
2. sts_model_3 + no sum_model : 54%
3. sts_model_3 + ext_sum_model : 45%
4. sts_model_4 + ext_sum_model : 53%
5. sts_model_3 + abs_sum_model : 45%
6. sts_model_4 + abs_sum_model : 54%
7. sts_model_5 + no sum_model : 65%
8. sts_model_5 + ext_sum_model : 65%
9. sts_model_5 + abs_sum_model : 64%
10. sts_model_6 + no sum_model : 59%
11. sts_model_6 + ext_sum_model : 56%
12. sts_model_7 + no sum_model : 57%
13. sts_model_7 + ext_sum_model : 54%
